In [1]:
from ai.lecture_notes.llama_predict import llama_promt
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from fuzzywuzzy import fuzz

from langdetect import detect
from nltk.stem import WordNetLemmatizer
from pymorphy2 import MorphAnalyzer
import nltk

from ai.transcriptor.Exctract_keywords import get_keywords, combined_stop_words
from ai.transcriptor.STT import speech2text
from ai.transcriptor.timecodes_and_full_text import timecodes_and_full_text

lang = 'Русский'

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nikolaibabuhin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
nltk.download('wordnet')

model_name = "facebook/nllb-200-distilled-600M"
model_t = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nikolaibabuhin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
d_langs_t = {'Английский': 'eng_Latn',
             'Датский': 'dan_Latn',
             'Испанский': 'spa_Latn',
             'Итальянский': 'ita_Latn',
             'Китайский': 'zho_Hans',
             'Немецкий': 'deu_Latn',
             'Польский': 'pol_Latn',
             'Португальский': 'por_Latn',
             'Турецкий': 'tur_Latn',
             'Французский': 'fra_Latn',
             'Чешский': 'ces_Latn',
             'Русский': 'rus_Cyrl'
             # 'Японский': 'jpn_Jpan',
             }

In [4]:
def lemmatize_word(word):
    # Определяем язык слова
    language = detect(word)

    if language == "en":
        lemmatizer = WordNetLemmatizer()
        return lemmatizer.lemmatize(word)
    elif language == "ru":
        morph = MorphAnalyzer()
        return morph.parse(word)[0].normal_form
    else:
        return word  # Если язык не определен или не поддерживается

In [5]:

# Define a function to remove similar words
def remove_similar_words(keywords):
    import pymorphy2
    morph = pymorphy2.MorphAnalyzer()
    unique_keywords = []
    for keyword in keywords:
        is_similar = False
        for unique_keyword in unique_keywords:
            if fuzz.ratio(keyword[0], unique_keyword[0]) > 70:
                is_similar = True
                break
        if not is_similar:
            #teg = morph.parse(keyword[0])[0]
            unique_keywords.append((lemmatize_word(keyword[0]), keyword[1]))
    return unique_keywords

In [6]:
def translator(text, target_language):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model_t.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[target_language])
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)


In [7]:
def sound_to_text(audio_file):
    lang = 'Русский'
    conspect = timecodes_and_full_text(audio_file)
    """
    С тайм кодами разбивка + конспект лекции
    timecode_with_text': [[x['start'], x['end'], x['text']] for x in data['segments']],
        'text': data['text']
    """
    print(conspect)
    text = conspect['text']
    termins = get_keywords(text, stopwords=combined_stop_words, keyphrase_ngram_range=(1, 1), top_n=25)
    # Remove similar words
    termins = remove_similar_words(termins)
    print(termins)
    """[('python', 0.4077), ('программирования', 0.31), ('разработчика', 0.2249)]"""
    out = []
    """for x in termins:
        if len(x[0]) < 499:
            out.append(translator(llama_promt(x[0]), d_langs_t.get(lang, 'rus_Cyrl')))
        else:
            print("превышен порог в 500 символов")"""
    #print(out)
    return out, conspect, termins

In [8]:
import pandas as pd

# Создайте DataFrame с данными для каждой колонки
data = {
    'File': [],
    'Term': []
}

# Создайте DataFrame из словаря данных
df = pd.DataFrame(data)

# Запишите DataFrame в CSV файл
df.to_csv('new_test.csv', index=False)

In [ ]:
import os

# Укажите путь к папке, в которой находятся файлы
folder_path = './new_test/audiofiles'

# Получите список всех файлов в папке
file_list = os.listdir(folder_path)

# Переберите каждый файл в списке
for file_name in file_list:
    # Проверьте, является ли элемент путем к файлу
    if os.path.isfile(os.path.join(folder_path, file_name)):
        # Выведите ссылку на файл
        print(os.path.join(folder_path, file_name))
        out, conspect, termins = sound_to_text(os.path.join(folder_path, file_name))
        termins
        for x in termins:
                # Создайте новую строку, которую хотите добавить
            '''new_row = {'File': folder_path,
                       'Term': translator(llama_promt(x[0]), d_langs_t.get(lang, 'rus_Cyrl'))}'''
            new_row = {'File': folder_path,
                       'Term': translator(x[0], d_langs_t.get(lang, 'rus_Cyrl'))}    
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)    

./new_test/audiofiles/audio3.mp3
Аудиофайл разбит на сегменты


/Users/nikolaibabuhin/PycharmProjects/geekbrains-ai-assistant/venv/lib/python3.9/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/nikolaibabuhin/PycharmProjects/geekbrains-ai-assistant/venv/lib/python3.9/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/nikolaibabuhin/PycharmProjects/geekbrains-ai-assistant/venv/lib/python3.9/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/nikolaibabuhin/PycharmProjects/geekbrains-ai-assistant/venv/lib/python3.9/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 in

{'timecode_with_text': [[0.0, 6.08, ' Тест может пережить рефакторинг тестируемого нами кода. То есть что такое рефакторинг?'], [6.08, 13.92, ' Рефакторинг – это изменение деталей имплементации без изменения наблюдаемого поведения. А простота'], [13.92, 20.7, ' поддержки оценивает затраты на сопровождение кода. На слайде показано взаимодействие пользователя с'], [20.7, 26.84, ' тестируемой системой. Пользователь вводит какие-то данные, система производит необходимые'], [26.84, 34.36, ' вычисления. Это шаг 1, шаг 2 и шаг 3 и возвращает пользователю какой-то результат. Давайте'], [34.36, 40.2, ' рассмотрим как могла бы выглядеть такая схема взаимодействия на примере нашего класса'], [40.2, 47.120000000000005, ' калькулятор из прошлой лекции. Пользователь приходит, передает метод операнды и оператор'], [47.120000000000005, 54.0, ' на первом шаге. На втором шаге метод calculation производит необходимые вычисления, а на третьем'], [54.0, 60.28, ' шаге метод возвращает пользователю результат

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Аудиофайл разбит на сегменты


/Users/nikolaibabuhin/PycharmProjects/geekbrains-ai-assistant/venv/lib/python3.9/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/nikolaibabuhin/PycharmProjects/geekbrains-ai-assistant/venv/lib/python3.9/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, y

{'timecode_with_text': [[0.0, 4.0, ' Далее у нас подрядные услуги коммерческому заказчику'], [4.0, 6.48, ' и подрядные услуги для госсектора.'], [6.48, 10.08, ' Если вам передали внутренний проект, то есть стейкхолдерами'], [10.08, 14.66, ' являются представители вашей организации, договорной'], [14.66, 16.76, ' работы как таковой не будет.'], [16.76, 20.18, ' Вместо договора у вас будет устав, также предполагается'], [20.18, 22.92, ' соблюдение политик и процедур компании.'], [22.92, 25.76, ' Исключением являются случаи, при которых вам необходимо'], [25.76, 27.88, ' привлечь внешнюю экспертизу.'], [27.88, 32.04, ' Что означает, что договорную работу вы будете вести'], [32.04, 33.72, ' на уже с подрядчиком.'], [33.72, 36.04, ' Подрядные услуги коммерческому заказчику.'], [36.04, 40.2, ' Большая часть лекций посвящена именно этому формату работы.'], [40.2, 42.92, ' Вы с проектной командой осуществляете и те услуги'], [42.92, 44.76, ' для внешнего заказчика.'], [44.76, 47.8, ' Началом п

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Аудиофайл разбит на сегменты


/Users/nikolaibabuhin/PycharmProjects/geekbrains-ai-assistant/venv/lib/python3.9/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/nikolaibabuhin/PycharmProjects/geekbrains-ai-assistant/venv/lib/python3.9/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, y

In [ ]:
df.to_csv("new_test1.csv")